## Redfin Web Scraping Project

### Import libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
from time import sleep
import smtplib


In [2]:
## Import Selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [4]:
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://www.redfin.ca/bc/vancouver')

C:\Users\markm\AppData\Local\Temp\ipykernel_57804\1989050257.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


### Navigating using Selenium

In [5]:
## Note: use this when the driver goes to default main page

In [6]:
search_bar = driver.find_element(By.XPATH,'//*[@id="search-box-input"]')

In [7]:
search_bar.send_keys('Vancouver, BC')

In [8]:
search_bar.send_keys(Keys.RETURN)

### Parsing with BeautifulSoup

In [9]:
#NOTE for scrolling down - run when necessary
#scrolling down
# driver.execute_script("window.scrollTo(0, window.scrollY + 250)")

#### Get Information about Properties in Greater Vancouver Area

In [ ]:
price_list = []

for price in soup.find_all('span', {'class': 'homecardV2Price'}):
    price_list.append(price.get_text())
price_list

In [ ]:
address_list = []

for address in soup.find_all('div', {'class': 'link-and-anchor'}):
    address_list.append(address.get_text())
address_list 

In [ ]:
home_stat = []

for stat in soup.find_all('div', {'class': 'HomeStatsV2 font-size-small'}):
    home_stat.append(stat.get_text())
home_stat

In [ ]:
## Looping to pages to get information about properties

In [ ]:
## get number of pages each city name
## split by comma and strip the item in the listing
def get_number_of_pages(city_name):
    formatted_city_name = city_name.split(',')
    formatted_province = formatted_city_name[1].strip().lower()
    formatted_city = formatted_city_name[0].strip().lower()
    url = f'https://www.redfin.ca/{formatted_province}/{formatted_city}'
    header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'referer': 'https://www.redfin.ca/'
             }
    html = requests.get(url=url,headers=header)
    soup = BeautifulSoup(html.content, 'lxml')
    no_pages = int(soup.find('div', {'class': 'viewingPage'}).get_text().split()[4])
    return no_pages

In [ ]:
#get page url
def get_page_url(city_name):
    formatted_city_name = city_name.split(',')
    formatted_province = formatted_city_name[1].strip().lower()
    formatted_city = formatted_city_name[0].strip().lower()
    test=[]
    for page_number in range(0, get_number_of_pages(city_name)+1):
        url = f'https://www.redfin.ca/{formatted_province}/{formatted_city}/page-{page_number}'
        header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
        'referer': 'https://www.redfin.ca/'
             }
        test.append(url)
    return test

In [ ]:
get_number_of_pages('Vancouver, BC')
get_page_url('Vancouver, BC')[5]

In [ ]:
#get properties listing
def property_listing(city_name):
    
#    driver.get(url)
    for page_number in range(1, get_number_of_pages(city_name)+1):
        print(page_number)
        print(get_page_url(city_name)[page_number])
        url = get_page_url(city_name)[page_number]
        driver.get(url)
        
        result = requests.get(url)
        soup = BeautifulSoup(result.content)
        
        data = {
            'price_list': [],
            'address': [],
            'home_stat': []
        }
        for listing in listings
    

In [ ]:
property_listing('Vancouver, BC')

#### User defines city name and province of the area they want to scrape

In [ ]:
city_name_list = ['Vancouver, BC', 'Toronto, ON']

In [ ]:
## Main loop for scraping
for index, city_name in enumerate(city_name_list):
    property_listing(city_name)